In [1]:
import torch

In [2]:
x = torch.empty(5,3)
x

tensor([[1.0986e-24, 4.5796e-41, 1.0360e-24],
        [4.5796e-41, 1.0361e-24, 4.5796e-41],
        [1.0451e-24, 4.5796e-41, 1.0009e-24],
        [4.5796e-41, 9.5415e-25, 4.5796e-41],
        [9.7026e-25, 4.5796e-41, 9.7038e-25]])

In [4]:
x = torch.rand(5,3)
x

tensor([[0.0168, 0.3407, 0.3221],
        [0.6120, 0.0057, 0.1294],
        [0.6823, 0.0903, 0.1984],
        [0.4241, 0.1888, 0.4807],
        [0.1142, 0.7430, 0.1228]])

In [5]:
torch.zeros(5,3,dtype=torch.long)

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [6]:
x = x.new_ones(5,3,dtype = torch.double)
x

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)

In [7]:
x.size()

torch.Size([5, 3])

In [8]:
y = torch.rand(5,3)
x + y

tensor([[1.1679, 1.4250, 1.5143],
        [1.5836, 1.1470, 1.8971],
        [1.6346, 1.6546, 1.1662],
        [1.2857, 1.4555, 1.8977],
        [1.0205, 1.5446, 1.9860]], dtype=torch.float64)

In [9]:
torch.add(x,y)

tensor([[1.1679, 1.4250, 1.5143],
        [1.5836, 1.1470, 1.8971],
        [1.6346, 1.6546, 1.1662],
        [1.2857, 1.4555, 1.8977],
        [1.0205, 1.5446, 1.9860]], dtype=torch.float64)

In [10]:
x[:,1]

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

In [12]:
x = torch.randn(4,4)
y = x.view(16)
z = x.view(-1,8)
print(x.size(),y.size(),z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


# Autograd 机制

In [14]:
x = torch.randn(3,4,requires_grad=True)
x

tensor([[ 0.3279, -0.5623,  0.5836,  0.6878],
        [-0.2983, -1.3592,  0.3576,  1.4544],
        [-0.7324, -0.3933, -0.9253,  2.2325]], requires_grad=True)

In [16]:
b = torch.randn(3,4,requires_grad=True)
b

tensor([[-0.2721,  0.4063,  0.7116, -1.1060],
        [ 0.1950,  0.4343,  1.3776,  0.2678],
        [-0.8876, -0.4481, -1.4635, -1.1827]], requires_grad=True)

In [19]:
t = x+b
t

tensor([[ 0.0557, -0.1560,  1.2952, -0.4182],
        [-0.1034, -0.9249,  1.7352,  1.7222],
        [-1.6201, -0.8413, -2.3889,  1.0498]], grad_fn=<AddBackward0>)

In [18]:
y = t.sum()
y

tensor(-0.5945, grad_fn=<SumBackward0>)

In [23]:
q = t.new_ones(5,3,dtype = torch.double)
q.sum()

tensor(15., dtype=torch.float64)

In [25]:
y.backward()
b.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [26]:
x.requires_grad,b.requires_grad,t.requires_grad

(True, True, True)

In [27]:
x = torch.rand(1)
b = torch.rand(1,requires_grad = True)
w = torch.rand(1,requires_grad=True)
y = w*x
z = y+b

In [28]:
x.is_leaf

True

In [29]:
z.backward(retain_graph=True) # 如果不清空会累加起来
w.grad

tensor([0.4310])

In [30]:
b.grad

tensor([1.])

# 线性回归模型

In [32]:
# 构造一组输入数据x和对应的标签y
import numpy as np
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1,1)
x_train.shape

(11, 1)

In [33]:
y_values = [2*i for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1,1)
y_train.shape

(11, 1)

In [37]:
x_train

array([[ 0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.]], dtype=float32)

In [39]:
# 线性回归就是一个不加激活函数的全连接层
import torch.nn as nn
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel,self).__init__()
        self.linear = nn.Linear(input_dim,output_dim)
    
    def forward(self,x):
        out = self.linear(x)
        return out

In [40]:
input_dim = 1
output_dim =1
model = LinearRegressionModel(input_dim,output_dim)

In [41]:
model

LinearRegressionModel(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [42]:
# 指定好参数和损失函数
epochs = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)
criterion = nn.MSELoss()

In [45]:
for epoch in range(epochs):
    epoch+=1
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)
    
    optimizer.zero_grad()
    
    outputs = model(inputs)
    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()
    if epoch%50 == 0:
        print('epoch {}, loss{}'.format(epoch,loss.item()))

epoch 50, loss0.11394837498664856
epoch 100, loss0.06499189138412476
epoch 150, loss0.037068963050842285
epoch 200, loss0.02114272490143776
epoch 250, loss0.012059038504958153
epoch 300, loss0.006878036540001631
epoch 350, loss0.003922977950423956
epoch 400, loss0.0022375064436346292
epoch 450, loss0.0012761916732415557
epoch 500, loss0.0007278912235051394
epoch 550, loss0.00041515950579196215
epoch 600, loss0.0002367933193454519
epoch 650, loss0.00013505984679795802
epoch 700, loss7.703189476160333e-05
epoch 750, loss4.3935924622928724e-05
epoch 800, loss2.5058217943296768e-05
epoch 850, loss1.4293175809143577e-05
epoch 900, loss8.152200280164834e-06
epoch 950, loss4.649482434615493e-06
epoch 1000, loss2.65217181549815e-06


In [46]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[3.0293709e-03],
       [2.0025933e+00],
       [4.0021572e+00],
       [6.0017209e+00],
       [8.0012846e+00],
       [1.0000849e+01],
       [1.2000413e+01],
       [1.3999976e+01],
       [1.5999540e+01],
       [1.7999104e+01],
       [1.9998667e+01]], dtype=float32)

# Tensor常见的形式有哪些
- 0:scalar
- 1:vector
- 2:matrix
- 3:n-dimensional tensor

In [ ]:
#scalar 是一个数值